# **scNET: Learning Context-Specific Gene and Cell Embeddings by Integrating Single-Cell Gene Expression Data with Protein-Protein Interaction Information**

## **Creating the workspace**

**Mount your drive**

In [4]:
from google.colab import drive
drive.mount("/content/gdrive")

PATH = "/content/gdrive/MyDrive"
%cd $PATH

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive/MyDrive


**Clone GitHub repository**


In [8]:
#!git clone https://github.com/madilabcode/scNET
%cd "./scNET"

/content/gdrive/MyDrive/scNET


**Download example data - as an h5ad object**

In [ ]:
import gdown
download_url = f'https://drive.google.com/uc?id=1YxQL1_BcbeAqVS7KhqzFzIJIyZjtLCcG'
output_path = './Data/example.h5ad'
gdown.download(download_url, output_path, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1YxQL1_BcbeAqVS7KhqzFzIJIyZjtLCcG
From (redirected): https://drive.google.com/uc?id=1YxQL1_BcbeAqVS7KhqzFzIJIyZjtLCcG&confirm=t&uuid=19763705-368a-437c-b0fe-f41dc2af6796
To: /content/gdrive/MyDrive/scNET/Data/example.h5ad
100%|██████████| 967M/967M [00:09<00:00, 98.0MB/s]


'./Data/example.h5ad'

**Downlaod Packages**

In [2]:
# Add this in a Google Colab cell to install the correct version of Pytorch Geometric.
import torch

def format_pytorch_version(version):
  return version.split('+')[0]

TORCH_version = torch.__version__
TORCH = format_pytorch_version(TORCH_version)

def format_cuda_version(version):
  return 'cu' + version.replace('.', '')

CUDA_version = torch.version.cuda
CUDA = format_cuda_version(CUDA_version)

#!pip install torch-scatter     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
#!pip install torch-sparse      -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
#!pip install torch-cluster     -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
#!pip install torch-spline-conv -f https://pytorch-geometric.com/whl/torch-{TORCH}+{CUDA}.html
!pip install torch-geometric
!pip install scanpy
!pip install mygene
!pip install gseapy
!pip install matplotlib datashader bokeh holoviews scikit-image colorcet igraph leidenalg

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 29.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.4/122.4 kB 17.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 9.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 12.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.8/79.8 kB 10.5 MB/s eta 0:00:00
  Created wheel for session-info: filename=session_info-1.0.0-py3-none-any.whl size=8026 sha256=9ae9ec6c512c70923dd3792aa85131f764445c0b8459ce63b76f929964becfc0
  Stored in directory: /root/.cache/pip/wheels/6a/aa/b9/eb5d4031476ec10802795b97ccf937b9bd998d68a9b268765a
Successfully built session-info
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 552.9/552.9 kB 15.0 MB/s eta 0:00:00


**Imports**

In [ ]:
import os
import torch
from main import main
import warnings
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
torch.manual_seed(42)

## **Train the model - the model, embeddings, and KNN will be saved**


### **The following parameters need to be provided to the main function in order to start training the model:**

**path:** The directory for the h5ad object.

**human_flag:** Is the data from human (True) or mouse (False)?

**pre_processing_flag:** Should a basic pre-processing pipeline be applied to the object?

**number_of_batches:** The number of batches for the training.

**split_cells:** If the number of cells is too large, we should split the cells into subsets to reduce GPU memory consumption. If true, the cells will be split.

**model_name:** A unique name for the project, which will be used in the outputs of the model.

### **The function will save the following:**

**cell_embedding:** A new dense representation of the cells under the Embedding folder.

**gene_embedding:** A new dense representation of the genes under the Embedding folder.

**trained_model:** The complete trained model under the Models folder.

**new_knn:** The new and pruned KNN network, but only if the split_cells flag is set to False.

In [ ]:
main(path = r"./Data/example.h5ad", pre_processing_flag=False, human_flag=True,number_of_batches=10, split_cells= True, max_epoch=50, model_name = "_example")

**Example of using the gene embedding for creating co-embedded network and performing KEGG prediction**

**Load all the relevant embeddings**

In [ ]:
import coEmbeddedNetwork as cen
gene_embedding, cell_embedding, node_feature = cen.load_embeddings("_example")

**We can plot the genes UMAP**

In [ ]:
cen.plot_gene_umap_clustring(gene_embedding)

**Creating the co-embedded network, is it modular?**

In [ ]:
graph, mod = cen.build_co_embeded_network(gene_embedding, node_feature)
mod

**Now we can use the network to predict KEGG pathway membership in a cross-validation**

In [ ]:
cen.predict_kegg(gene_embedding,node_feature)